In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,classification_report,f1_score,plot_confusion_matrix,roc_auc_score,roc_curve,ConfusionMatrixDisplay
from sklearn.metrics import plot_roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
import math
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv(r'C:\MLCourse\Learning\Guvi_Final_Projects\bank-additional-full.csv',delimiter=';')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
df_2 = df.copy()
df_2['job'] = df_2['job'].map({'admin.':1, 'blue-collar':2, 'entrepreneur':3, 'housemaid':4, 'management':5, 'retired':6, 'self-employed':7, 'services':8, 'student':9, 'technician':10, 'unemployed':11, 'unknown':12})
df_2['marital'] = df_2['marital'].map({'divorced':1, 'married':2, 'single':3, 'unknown':4})
df_2['education'] = df_2['education'].map({'basic.4y':1, 'basic.6y':2, 'basic.9y':3, 'high.school':4, 'illiterate':5, 'professional.course':6, 'university.degree':7, 'unknown':8})
df_2['default'] = df_2['default'].map({'no':1, 'unknown':2, 'yes':3})
df_2['housing'] = df_2['housing'].map({'no':1, 'unknown':2, 'yes':3})
df_2['loan'] = df_2['loan'].map({'no':1, 'unknown':2, 'yes':3})
df_2['contact'] = df_2['contact'].map({'cellular':1, 'telephone':2})
df_2['month'] = df_2['month'].map({'may':5, 'jun':6, 'jul':7, 'aug':8, 'oct':10, 'nov':11, 'dec':12,
                                   'mar':3, 'apr':4,'sep':9})
df_2['day_of_week'] = df_2['day_of_week'].map({'mon':1, 'tue':2, 'wed':3, 'thu':4, 'fri':5})
df_2['poutcome'] = df_2['poutcome'].map({'failure':1, 'nonexistent':2, 'success':3})
df_2['y'] = df_2['y'].map({'no':0,'yes':1})

In [4]:
df_2.shape

(41188, 21)

In [9]:
new_df = df_2.sample(300)
new_df.shape

(300, 21)

In [10]:
new_df.groupby('y').size()

y
0    264
1     36
dtype: int64

In [12]:
y = new_df['y']
X = new_df.drop('y', axis=1)
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X,y)

y_sm.value_counts()

0    264
1    264
Name: y, dtype: int64

In [13]:
def disp_f1_score_sampling(X,y):
    x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42,stratify=y)
    model = SVC()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(classification_report(y_test,y_pred))

In [14]:
print('\t \tClassification report for the SMOTE dataset')
print('')
disp_f1_score_sampling(X_sm,y_sm)

	 	Classification report for the SMOTE dataset

              precision    recall  f1-score   support

           0       0.71      0.91      0.80        66
           1       0.88      0.64      0.74        66

    accuracy                           0.77       132
   macro avg       0.79      0.77      0.77       132
weighted avg       0.79      0.77      0.77       132



In [15]:
ss = StandardScaler()
X_ss = ss.fit_transform(X_sm)
X_train_ss, X_test_ss, y_train_ss, y_test_ss = train_test_split(X_ss,y_sm,test_size=0.2, random_state=42)
model_ss = SVC()
model_ss.fit(X_train_ss,y_train_ss)
y_pred_ss = model_ss.predict(X_test_ss)
print('Classification report of SMOTE-scaled dataset')
print(classification_report(y_test_ss,y_pred_ss))

Classification report of SMOTE-scaled dataset
              precision    recall  f1-score   support

           0       0.98      0.96      0.97        57
           1       0.96      0.98      0.97        49

    accuracy                           0.97       106
   macro avg       0.97      0.97      0.97       106
weighted avg       0.97      0.97      0.97       106



In [2]:
pipeline1 = make_pipeline(LogisticRegression(solver='liblinear'))
pipeline2 = make_pipeline(SVC())
pipeline3 = make_pipeline(KNeighborsClassifier(n_jobs=-1))
pipeline4 = make_pipeline(BaggingClassifier(n_jobs=-1))
pipeline5 = make_pipeline(GradientBoostingClassifier())
pipeline7 = make_pipeline(RandomForestClassifier(n_jobs=-1))

In [18]:
alphas = [0.5,1,1.5,2,2.5,3,3.5,4,4.5]
penalty = ['l1', 'l2']
max_iter = [10,20,50,100,150,200,500,1000]

params_1 = {
    'logisticregression__C': alphas,
    'logisticregression__penalty': penalty,
     
    'logisticregression__max_iter': max_iter
}
grid1 = GridSearchCV(pipeline1, param_grid=params_1, cv=5,n_jobs=1, verbose=4)
grid1.fit(X,y)
print('For Logistic Regression')
print('Best Paramaters')
print(grid1.best_params_)
print('Best score')
print(grid1.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV 1/5] END logisticregression__C=0.5, logisticregression__max_iter=10, logisticregression__penalty=l1;, score=0.900 total time=   0.0s
[CV 2/5] END logisticregression__C=0.5, logisticregression__max_iter=10, logisticregression__penalty=l1;, score=0.900 total time=   0.0s
[CV 3/5] END logisticregression__C=0.5, logisticregression__max_iter=10, logisticregression__penalty=l1;, score=0.900 total time=   0.0s
[CV 4/5] END logisticregression__C=0.5, logisticregression__max_iter=10, logisticregression__penalty=l1;, score=0.917 total time=   0.0s
[CV 5/5] END logisticregression__C=0.5, logisticregression__max_iter=10, logisticregression__penalty=l1;, score=0.867 total time=   0.0s
[CV 1/5] END logisticregression__C=0.5, logisticregression__max_iter=10, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 2/5] END logisticregression__C=0.5, logisticregression__max_iter=10, logisticregression__penalty=l2;, score=0.8

[CV 1/5] END logisticregression__C=0.5, logisticregression__max_iter=500, logisticregression__penalty=l1;, score=0.917 total time=   0.4s
[CV 2/5] END logisticregression__C=0.5, logisticregression__max_iter=500, logisticregression__penalty=l1;, score=0.867 total time=   0.5s
[CV 3/5] END logisticregression__C=0.5, logisticregression__max_iter=500, logisticregression__penalty=l1;, score=0.900 total time=   0.3s
[CV 4/5] END logisticregression__C=0.5, logisticregression__max_iter=500, logisticregression__penalty=l1;, score=0.900 total time=   0.6s
[CV 5/5] END logisticregression__C=0.5, logisticregression__max_iter=500, logisticregression__penalty=l1;, score=0.883 total time=   0.4s
[CV 1/5] END logisticregression__C=0.5, logisticregression__max_iter=500, logisticregression__penalty=l2;, score=0.900 total time=   0.0s
[CV 2/5] END logisticregression__C=0.5, logisticregression__max_iter=500, logisticregression__penalty=l2;, score=0.867 total time=   0.0s
[CV 3/5] END logisticregression__C

[CV 2/5] END logisticregression__C=1, logisticregression__max_iter=150, logisticregression__penalty=l1;, score=0.883 total time=   0.5s
[CV 3/5] END logisticregression__C=1, logisticregression__max_iter=150, logisticregression__penalty=l1;, score=0.900 total time=   0.4s
[CV 4/5] END logisticregression__C=1, logisticregression__max_iter=150, logisticregression__penalty=l1;, score=0.917 total time=   0.7s
[CV 5/5] END logisticregression__C=1, logisticregression__max_iter=150, logisticregression__penalty=l1;, score=0.883 total time=   0.4s
[CV 1/5] END logisticregression__C=1, logisticregression__max_iter=150, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 2/5] END logisticregression__C=1, logisticregression__max_iter=150, logisticregression__penalty=l2;, score=0.867 total time=   0.0s
[CV 3/5] END logisticregression__C=1, logisticregression__max_iter=150, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 4/5] END logisticregression__C=1, logisticre

[CV 4/5] END logisticregression__C=1.5, logisticregression__max_iter=50, logisticregression__penalty=l1;, score=0.917 total time=   0.1s
[CV 5/5] END logisticregression__C=1.5, logisticregression__max_iter=50, logisticregression__penalty=l1;, score=0.883 total time=   0.2s
[CV 1/5] END logisticregression__C=1.5, logisticregression__max_iter=50, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 2/5] END logisticregression__C=1.5, logisticregression__max_iter=50, logisticregression__penalty=l2;, score=0.867 total time=   0.0s
[CV 3/5] END logisticregression__C=1.5, logisticregression__max_iter=50, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 4/5] END logisticregression__C=1.5, logisticregression__max_iter=50, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 5/5] END logisticregression__C=1.5, logisticregression__max_iter=50, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 1/5] END logisticregression__C=1.5, l

[CV 4/5] END logisticregression__C=2, logisticregression__max_iter=20, logisticregression__penalty=l1;, score=0.917 total time=   0.0s
[CV 5/5] END logisticregression__C=2, logisticregression__max_iter=20, logisticregression__penalty=l1;, score=0.883 total time=   0.0s
[CV 1/5] END logisticregression__C=2, logisticregression__max_iter=20, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 2/5] END logisticregression__C=2, logisticregression__max_iter=20, logisticregression__penalty=l2;, score=0.900 total time=   0.0s
[CV 3/5] END logisticregression__C=2, logisticregression__max_iter=20, logisticregression__penalty=l2;, score=0.900 total time=   0.0s
[CV 4/5] END logisticregression__C=2, logisticregression__max_iter=20, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 5/5] END logisticregression__C=2, logisticregression__max_iter=20, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 1/5] END logisticregression__C=2, logisticregressio

...

[CV 1/5] END logisticregression__C=3, logisticregression__max_iter=150, logisticregression__penalty=l1;, score=0.883 total time=   0.7s
[CV 2/5] END logisticregression__C=3, logisticregression__max_iter=150, logisticregression__penalty=l1;, score=0.883 total time=   1.0s
[CV 3/5] END logisticregression__C=3, logisticregression__max_iter=150, logisticregression__penalty=l1;, score=0.917 total time=   0.8s
[CV 4/5] END logisticregression__C=3, logisticregression__max_iter=150, logisticregression__penalty=l1;, score=0.917 total time=   1.3s
[CV 5/5] END logisticregression__C=3, logisticregression__max_iter=150, logisticregression__penalty=l1;, score=0.883 total time=   0.5s
[CV 1/5] END logisticregression__C=3, logisticregression__max_iter=150, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 2/5] END logisticregression__C=3, logisticregression__max_iter=150, logisticregression__penalty=l2;, score=0.867 total time=   0.0s
[CV 3/5] END logisticregression__C=3, logisticre

[CV 2/5] END logisticregression__C=3.5, logisticregression__max_iter=50, logisticregression__penalty=l1;, score=0.867 total time=   0.2s
[CV 3/5] END logisticregression__C=3.5, logisticregression__max_iter=50, logisticregression__penalty=l1;, score=0.917 total time=   0.1s
[CV 4/5] END logisticregression__C=3.5, logisticregression__max_iter=50, logisticregression__penalty=l1;, score=0.917 total time=   0.0s
[CV 5/5] END logisticregression__C=3.5, logisticregression__max_iter=50, logisticregression__penalty=l1;, score=0.883 total time=   0.2s
[CV 1/5] END logisticregression__C=3.5, logisticregression__max_iter=50, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 2/5] END logisticregression__C=3.5, logisticregression__max_iter=50, logisticregression__penalty=l2;, score=0.867 total time=   0.0s
[CV 3/5] END logisticregression__C=3.5, logisticregression__max_iter=50, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 4/5] END logisticregression__C=3.5, l

[CV 4/5] END logisticregression__C=4, logisticregression__max_iter=20, logisticregression__penalty=l1;, score=0.917 total time=   0.0s
[CV 5/5] END logisticregression__C=4, logisticregression__max_iter=20, logisticregression__penalty=l1;, score=0.883 total time=   0.0s
[CV 1/5] END logisticregression__C=4, logisticregression__max_iter=20, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 2/5] END logisticregression__C=4, logisticregression__max_iter=20, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 3/5] END logisticregression__C=4, logisticregression__max_iter=20, logisticregression__penalty=l2;, score=0.900 total time=   0.0s
[CV 4/5] END logisticregression__C=4, logisticregression__max_iter=20, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 5/5] END logisticregression__C=4, logisticregression__max_iter=20, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 1/5] END logisticregression__C=4, logisticregressio

[CV 5/5] END logisticregression__C=4, logisticregression__max_iter=1000, logisticregression__penalty=l1;, score=0.883 total time=   0.9s
[CV 1/5] END logisticregression__C=4, logisticregression__max_iter=1000, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 2/5] END logisticregression__C=4, logisticregression__max_iter=1000, logisticregression__penalty=l2;, score=0.867 total time=   0.0s
[CV 3/5] END logisticregression__C=4, logisticregression__max_iter=1000, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 4/5] END logisticregression__C=4, logisticregression__max_iter=1000, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 5/5] END logisticregression__C=4, logisticregression__max_iter=1000, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 1/5] END logisticregression__C=4.5, logisticregression__max_iter=10, logisticregression__penalty=l1;, score=0.900 total time=   0.0s
[CV 2/5] END logisticregression__C=4.5, l

[CV 5/5] END logisticregression__C=4.5, logisticregression__max_iter=200, logisticregression__penalty=l1;, score=0.883 total time=   0.9s
[CV 1/5] END logisticregression__C=4.5, logisticregression__max_iter=200, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 2/5] END logisticregression__C=4.5, logisticregression__max_iter=200, logisticregression__penalty=l2;, score=0.867 total time=   0.0s
[CV 3/5] END logisticregression__C=4.5, logisticregression__max_iter=200, logisticregression__penalty=l2;, score=0.900 total time=   0.0s
[CV 4/5] END logisticregression__C=4.5, logisticregression__max_iter=200, logisticregression__penalty=l2;, score=0.917 total time=   0.0s
[CV 5/5] END logisticregression__C=4.5, logisticregression__max_iter=200, logisticregression__penalty=l2;, score=0.883 total time=   0.0s
[CV 1/5] END logisticregression__C=4.5, logisticregression__max_iter=500, logisticregression__penalty=l1;, score=0.883 total time=   1.9s
[CV 2/5] END logisticregression__C

In [ ]:
For Logistic Regression
Best Paramaters
{'logisticregression__C': 0.5, 'logisticregression__max_iter': 20, 'logisticregression__penalty': 'l2'}
Best score
0.9033333333333333

In [20]:
C = [1,2,3,4,5,10]
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
degree = [2,3,4,5]

params_2 = {
            "svc__C":C,
            "svc__kernel":kernel,
            "svc__degree":degree 
    
}
grid2 = GridSearchCV(pipeline2, param_grid=params_2, cv=5,n_jobs=1, verbose=3)
grid2.fit(X,y)
print('For SVC')
print('Best Paramaters')
print(grid2.best_params_)
print('Best score')
print(grid2.best_score_)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END svc__C=1, svc__degree=2, svc__kernel=linear;, score=0.867 total time=   0.5s
[CV 2/5] END svc__C=1, svc__degree=2, svc__kernel=linear;, score=0.867 total time=   1.0s
[CV 3/5] END svc__C=1, svc__degree=2, svc__kernel=linear;, score=0.933 total time=   0.3s
[CV 4/5] END svc__C=1, svc__degree=2, svc__kernel=linear;, score=0.900 total time=   0.2s
[CV 5/5] END svc__C=1, svc__degree=2, svc__kernel=linear;, score=0.883 total time=   0.6s
[CV 1/5] END svc__C=1, svc__degree=2, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 2/5] END svc__C=1, svc__degree=2, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 3/5] END svc__C=1, svc__degree=2, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 4/5] END svc__C=1, svc__degree=2, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 5/5] END svc__C=1, svc__degree=2, svc__kernel=poly;, score=0.867 total time=   0.0s
[CV 1/5] END svc__C=1, svc__degree=2, svc__kerne

[CV 1/5] END svc__C=2, svc__degree=3, svc__kernel=linear;, score=0.900 total time=   0.3s
[CV 2/5] END svc__C=2, svc__degree=3, svc__kernel=linear;, score=0.850 total time=   0.4s
[CV 3/5] END svc__C=2, svc__degree=3, svc__kernel=linear;, score=0.917 total time=   0.3s
[CV 4/5] END svc__C=2, svc__degree=3, svc__kernel=linear;, score=0.900 total time=   0.7s
[CV 5/5] END svc__C=2, svc__degree=3, svc__kernel=linear;, score=0.900 total time=   1.0s
[CV 1/5] END svc__C=2, svc__degree=3, svc__kernel=poly;, score=0.900 total time=   0.0s
[CV 2/5] END svc__C=2, svc__degree=3, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 3/5] END svc__C=2, svc__degree=3, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 4/5] END svc__C=2, svc__degree=3, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 5/5] END svc__C=2, svc__degree=3, svc__kernel=poly;, score=0.867 total time=   0.0s
[CV 1/5] END svc__C=2, svc__degree=3, svc__kernel=rbf;, score=0.883 total time=   0.0s
[CV 2/5] END svc__C=2, 

[CV 1/5] END svc__C=3, svc__degree=4, svc__kernel=linear;, score=0.900 total time=   0.3s
[CV 2/5] END svc__C=3, svc__degree=4, svc__kernel=linear;, score=0.850 total time=   0.5s
[CV 3/5] END svc__C=3, svc__degree=4, svc__kernel=linear;, score=0.917 total time=   0.3s
[CV 4/5] END svc__C=3, svc__degree=4, svc__kernel=linear;, score=0.900 total time=   1.3s
[CV 5/5] END svc__C=3, svc__degree=4, svc__kernel=linear;, score=0.917 total time=   0.8s
[CV 1/5] END svc__C=3, svc__degree=4, svc__kernel=poly;, score=0.917 total time=   0.0s
[CV 2/5] END svc__C=3, svc__degree=4, svc__kernel=poly;, score=0.850 total time=   0.0s
[CV 3/5] END svc__C=3, svc__degree=4, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 4/5] END svc__C=3, svc__degree=4, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 5/5] END svc__C=3, svc__degree=4, svc__kernel=poly;, score=0.867 total time=   0.0s
[CV 1/5] END svc__C=3, svc__degree=4, svc__kernel=rbf;, score=0.883 total time=   0.0s
[CV 2/5] END svc__C=3, 

[CV 1/5] END svc__C=4, svc__degree=5, svc__kernel=linear;, score=0.900 total time=   0.3s
[CV 2/5] END svc__C=4, svc__degree=5, svc__kernel=linear;, score=0.867 total time=   0.7s
[CV 3/5] END svc__C=4, svc__degree=5, svc__kernel=linear;, score=0.917 total time=   0.5s
[CV 4/5] END svc__C=4, svc__degree=5, svc__kernel=linear;, score=0.900 total time=   1.4s
[CV 5/5] END svc__C=4, svc__degree=5, svc__kernel=linear;, score=0.917 total time=   0.6s
[CV 1/5] END svc__C=4, svc__degree=5, svc__kernel=poly;, score=0.917 total time=   0.0s
[CV 2/5] END svc__C=4, svc__degree=5, svc__kernel=poly;, score=0.833 total time=   0.0s
[CV 3/5] END svc__C=4, svc__degree=5, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 4/5] END svc__C=4, svc__degree=5, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 5/5] END svc__C=4, svc__degree=5, svc__kernel=poly;, score=0.867 total time=   0.0s
[CV 1/5] END svc__C=4, svc__degree=5, svc__kernel=rbf;, score=0.883 total time=   0.0s
[CV 2/5] END svc__C=4, 

[CV 1/5] END svc__C=10, svc__degree=2, svc__kernel=linear;, score=0.917 total time=   0.5s
[CV 2/5] END svc__C=10, svc__degree=2, svc__kernel=linear;, score=0.850 total time=   0.4s
[CV 3/5] END svc__C=10, svc__degree=2, svc__kernel=linear;, score=0.917 total time=   0.7s
[CV 4/5] END svc__C=10, svc__degree=2, svc__kernel=linear;, score=0.900 total time=   1.6s
[CV 5/5] END svc__C=10, svc__degree=2, svc__kernel=linear;, score=0.917 total time=   0.8s
[CV 1/5] END svc__C=10, svc__degree=2, svc__kernel=poly;, score=0.917 total time=   0.0s
[CV 2/5] END svc__C=10, svc__degree=2, svc__kernel=poly;, score=0.833 total time=   0.0s
[CV 3/5] END svc__C=10, svc__degree=2, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 4/5] END svc__C=10, svc__degree=2, svc__kernel=poly;, score=0.883 total time=   0.0s
[CV 5/5] END svc__C=10, svc__degree=2, svc__kernel=poly;, score=0.867 total time=   0.0s
[CV 1/5] END svc__C=10, svc__degree=2, svc__kernel=rbf;, score=0.900 total time=   0.0s
[CV 2/5] END

In [21]:
n_neighbors = [3,5,7,9]
weights =['uniform', 'distance']
algorithm =['auto', 'ball_tree', 'kd_tree', 'brute']

params_3 = {'kneighborsclassifier__n_neighbors':n_neighbors,
            'kneighborsclassifier__weights':weights,
            'kneighborsclassifier__algorithm':algorithm
            }
grid3 = GridSearchCV(pipeline3, param_grid=params_3, cv=5,n_jobs=-1,verbose=3)
grid3.fit(X,y)
print('For KNN')
print('Best Paramaters')
print(grid3.best_params_)
print('Best score')
print(grid3.best_score_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
For KNN
Best Paramaters
{'kneighborsclassifier__algorithm': 'auto', 'kneighborsclassifier__n_neighbors': 5, 'kneighborsclassifier__weights': 'distance'}
Best score
0.8933333333333333


In [22]:
n_estimators = [5,10,15,20,25,30]
max_samples = [0.3,0.4,0.5,0.6,0.7,0.8]
max_features = [0.3,0.4,0.5,0.6,0.7,0.8]

params_4 = {
    'baggingclassifier__n_estimators':n_estimators,
    'baggingclassifier__max_samples':max_samples,
    'baggingclassifier__max_features':max_features
    }
grid4 = GridSearchCV(pipeline4, param_grid=params_4, cv=5,n_jobs=-1,verbose=3)
grid4.fit(X,y)
print('For Bagging Classifier')
print('Best Paramaters')
print(grid4.best_params_)
print('Best score')
print(grid4.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
For Bagging Classifier
Best Paramaters
{'baggingclassifier__max_features': 0.6, 'baggingclassifier__max_samples': 0.8, 'baggingclassifier__n_estimators': 15}
Best score
0.9133333333333333


In [23]:
loss =['deviance', 'exponential']
learning_rate = [0.01,0.05,0.1,0.15,0.2]
n_estimators = [10,15,20,25,30]
criterion =['friedman_mse', 'mse', 'mae']
min_samples_split = [2,5,7,9]
max_depth = [2,4,6,8]
min_impurity_decrease = [0.05,0.08,0.1]
max_features = ['auto', 'sqrt', 'log2']

params_5 = {
    'gradientboostingclassifier__loss':loss,
    'gradientboostingclassifier__learning_rate':learning_rate,
    'gradientboostingclassifier__n_estimators':n_estimators,
    'gradientboostingclassifier__criterion':criterion,
    'gradientboostingclassifier__min_samples_split':min_samples_split,
    'gradientboostingclassifier__max_depth':max_depth,
    'gradientboostingclassifier__min_impurity_decrease':min_impurity_decrease,
    'gradientboostingclassifier__max_features':max_features
    }
grid5 = GridSearchCV(pipeline5, param_grid=params_5, cv=5,n_jobs=-1,verbose=3)
grid5.fit(X,y)
print('For Gradient boosting classifier')
print('Best Paramaters')
print(grid5.best_params_)
print('Best score')
print(grid5.best_score_)

Fitting 5 folds for each of 21600 candidates, totalling 108000 fits
For Gradient boosting classifier
Best Paramaters
{'gradientboostingclassifier__criterion': 'friedman_mse', 'gradientboostingclassifier__learning_rate': 0.15, 'gradientboostingclassifier__loss': 'deviance', 'gradientboostingclassifier__max_depth': 4, 'gradientboostingclassifier__max_features': 'log2', 'gradientboostingclassifier__min_impurity_decrease': 0.08, 'gradientboostingclassifier__min_samples_split': 9, 'gradientboostingclassifier__n_estimators': 20}
Best score
0.9233333333333332


In [29]:
n_estimators = [50,75,100,125,150,200]
criterion = ["gini", "entropy"]
max_depth = [5,7,10,15,20]
max_features =["auto", "sqrt", "log2"]
min_impurity_decrease = [0.05,0.1,0.15,0.2]

params_7 = {
    'randomforestclassifier__n_estimators':n_estimators,
    'randomforestclassifier__criterion':criterion,
    'randomforestclassifier__max_depth':max_depth,
    'randomforestclassifier__max_features':max_features,
    'randomforestclassifier__min_impurity_decrease':min_impurity_decrease
        
}
grid7 = GridSearchCV(pipeline7, param_grid=params_7, cv=5,n_jobs=-1,verbose=3)
grid7.fit(X,y)
print('For RF classifier')
print('Best Paramaters')
print(grid7.best_params_)
print('Best score')
print(grid7.best_score_)


Fitting 5 folds for each of 720 candidates, totalling 3600 fits
For RF classifier
Best Paramaters
{'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_depth': 7, 'randomforestclassifier__max_features': 'log2', 'randomforestclassifier__min_impurity_decrease': 0.05, 'randomforestclassifier__n_estimators': 50}
Best score
0.8866666666666667
